# Немного кода для обкачивания интернета
## Модуль [requests](https://realpython.com/python-requests/)

Давайте посмотрим html формат

Лучший учебник - https://www.w3schools.com

Теперь скачаем html страничку

In [1]:
import requests

In [2]:
response = requests.get("https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/pervyy-v-rossii-detenysh-bolshoy-pandy/", verify=False)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.moscowzoo.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


В `response` теперь лежит ответ сервера. Это не просто html-код страницы, а еще дополнительная информация

In [5]:
response

<Response [200]>

In [6]:
response.status_code
# 200 - "всё работает"
# 404, 503 и др. - "произошла ошибка, не работает, такой страницы нет и т.д."

200

In [7]:
# Вывели первые 210 символов html
print(response.text[:210])

<!doctype html>
<html lang="ru">
<head>
	<meta name="yandex-verification" content="60483b355f0f5c1d" />
	<link rel="icon" type="image/x-icon" href="/local/templates/zoo/favicon.ico" />
		<meta http-equiv="Conte


Если сайт защищается от краулеров на питоне, можно, например, представиться Мозиллой:

In [8]:
url = 'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/pervyy-v-rossii-detenysh-bolshoy-pandy/'  # адрес страницы, которую мы хотим скачать
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером

response = requests.get(url, headers={'User-Agent':user_agent}, verify=False)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.moscowzoo.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Или использовать специальную библиотеку:

In [9]:
! pip install fake-useragent

In [10]:
from fake_useragent import UserAgent

In [11]:
user_agent = UserAgent().chrome
user_agent

'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

In [12]:
response = requests.get("https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/pervyy-v-rossii-detenysh-bolshoy-pandy/",
                        headers={'User-Agent':user_agent},
                        verify=False)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.moscowzoo.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [13]:
response.url  #ссылка

'https://www.moscowzoo.ru/about-zoo/news/novosti-zooparka/pervyy-v-rossii-detenysh-bolshoy-pandy/'

In [14]:
print(response.text[:300])  #html код

<!doctype html>
<html lang="ru">
<head>
	<meta name="yandex-verification" content="60483b355f0f5c1d" />
	<link rel="icon" type="image/x-icon" href="/local/templates/zoo/favicon.ico" />
		<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<script type="text/javascript" data-skip-mo


Существует несколько вариантов, как достать что-то из определенного тега, например, достать заголовок:

- регулярные выражения (плохой вариант)
- специальные библиотеки питона, например, BeautifulSoup (bs4) или lxml (хороший вариант)


In [15]:
from bs4 import BeautifulSoup, contents

In [17]:
# инициализируем (создаем) soup из response.text (кода страницы)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify()[:500])  # печатаем приукрашенный суп

<!DOCTYPE html>
<html lang="ru">
 <head>
  <meta content="60483b355f0f5c1d" name="yandex-verification"/>
  <link href="/local/templates/zoo/favicon.ico" rel="icon" type="image/x-icon"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script data-skip-moving="true" type="text/javascript">
   (function(w, d, n) {var cl = "bx-core";var ht = d.documentElement;var htc = ht ? ht.className : undefined;if (htc === undefined || htc.indexOf(cl) !== -1){return;}var ua = n.userAgen


Если мы хотим вывести заголовок, нужно посмотреть, какими тегами он оформляется.

Заходим на наш сайт

На этом конкретном сайте заголовки лежат внутри тегов `cp-banner`

In [25]:
name = soup.find('cp-banner')
print(name.prettify())

<cp-banner>
 <template #title="">
  ПЕРВЫЙ В РОССИИ ДЕТЕНЫШ БОЛЬШОЙ ПАНДЫ
 </template>
 <template #description="">
 </template>
</cp-banner>



А внутри этого раздела текст заголовка лежит внутри первого тега `template`

In [26]:
name2 = name.find('template')
print(name2.prettify())

<template #title="">
 ПЕРВЫЙ В РОССИИ ДЕТЕНЫШ БОЛЬШОЙ ПАНДЫ
</template>



Нас интересует только текст, поэтому сделаем `get_text`

In [ ]:
print(name2.get_text())

ПЕРВЫЙ В РОССИИ ДЕТЕНЫШ БОЛЬШОЙ ПАНДЫ


\- Почему мы не сделали сразу `soup.find('template')`?

\- Потому что тегов `template` несколько, а мы ищем уникальное обрамление нашего заголовка.

In [35]:
# Вот так мы попросили найти все теги template внутри cp-banner
soup.select('cp-banner > template')

[<template #title="">ПЕРВЫЙ В РОССИИ ДЕТЕНЫШ БОЛЬШОЙ ПАНДЫ</template>,
 <template #description=""></template>]

In [41]:
# Если мы уверены, что заголовок в первом из найденных тегов template,
# можем вывести первый тег
print(soup.select('cp-banner > template')[0])

# и его содержание
print('Содержание тега:')
print(soup.select('cp-banner > template')[0].get_text())

<template #title="">ПЕРВЫЙ В РОССИИ ДЕТЕНЫШ БОЛЬШОЙ ПАНДЫ</template>
Содержание тега:
ПЕРВЫЙ В РОССИИ ДЕТЕНЫШ БОЛЬШОЙ ПАНДЫ


А теперь найдём текст новости.

Он лежит в теге `<div class="dp-content-inner" style="color:#fff;">`

\- Как устроен этот тег?

\- div - имя тега, class и style - имена атрибутов, "dp-content-inner" и "color:#fff;" - значения атрибутов

In [43]:
post = soup.find('div', {'class':"dp-content-inner"})  # вот так можно указать атрибут тега
print(post.prettify())

<div class="dp-content-inner" style="color:#fff;">
 <div class="dp-content border-block">
  <div class="container">
   <div class="row">
    <div class="dp-content-inner" style="color:#fff;">
     В Московском зоопарке появился на свет первый в истории России детеныш большой панды. Это уникальное событие как для страны, так и для всего мирового природоохранного сообщества. Панды переданы Московскому зоопарку в рамках международной программы по сохранению, защите и исследованию больших панд. Миссия зоопарка – внести значимый вклад в создание резервной популяции этих редчайших животных.
     <br/>
     <br/>
     Малыш весом около 150 граммов родился у молодой пары больших панд — самца Жуи и самки Диндин. Пол детеныша пока неизвестен. Наша Диндин проявила большую заботу и сразу приняла малыша. За самкой круглосуточно наблюдают и берут все необходимые анализы.
     <br/>
     <br/>
     Первый в России детеныш большой панды появился на свет после спаривания естественным путём. Чтобы помоч

In [46]:
text = post.get_text()  # получили текст
print(text)






												В Московском зоопарке появился на свет первый в истории России детеныш большой панды. Это уникальное событие как для страны, так и для всего мирового природоохранного сообщества. Панды переданы Московскому зоопарку в рамках международной программы по сохранению, защите и исследованию больших панд. Миссия зоопарка – внести значимый вклад в создание резервной популяции этих редчайших животных. 
  
 Малыш весом около 150 граммов родился у молодой пары больших панд — самца Жуи и самки Диндин. Пол детеныша пока неизвестен. Наша Диндин проявила большую заботу и сразу приняла малыша. За самкой круглосуточно наблюдают и берут все необходимые анализы. 
  
 Первый в России детеныш большой панды появился на свет после спаривания естественным путём. Чтобы помочь коллегам, в Москву на три месяца прибыл эксперт по размножению больших панд из Исследовательского центра по сохранению больших панд Китая. Специалисты не торопили животных, четко следовали указаниям китайских коллег, добив

Вот здесь можно [посмотреть часть документации BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html)

## Знакомство с NER
### (Named-entity recognition, извлечение именованных сущностей)

Именнованные сущности: персоны, локации, организации (что ещё?)

BIOES-схема: к метке сущности (например, PER для персон или ORG для организаций) добавляется префикс, который обозначает позицию токена в спане сущности:

B – beginning – первый токен в спане сущности, которая состоит из нескольких токенов;

I – inside – внутри спана;

О – outside – токен не относится ни к какой сущности;

E – ending – последний токен сущности, которая состоит из нескольких токенов;

S – single – сущность состоит из одного токена.

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
! python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


2023-09-14 09:54:53.621817: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 09:54:54.995577: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 2.7 MB/s eta 0:00:00
  Obtaining dependency information for pymorphy3>=1.0.0 from https://files.pythonhosted.org/packages/d7/f9/ffb9afde503dc6bb2361ea79ceaea18138fbcee32aec4c5d8efa49180753/pymorphy3-1.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for docopt-ng>=0.6 from https://files.pythonhosted.org/packages/6c/4a/c3b77fc1a24510b08918b43a473410c0168f6e657118807015f1f1edceea/docopt_ng-0.9.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 52.8 MB/s

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("ru_core_news_sm")

In [ ]:
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Московском зоопарке 19 38 ORG
России 73 79 LOC
Московскому зоопарку 211 231 LOC
Диндин 505 511 PER
Диндин 548 554 PER
России 686 692 LOC
Москву 794 800 LOC
Исследовательского центра по сохранению больших панд 861 913 ORG
Китая 914 919 LOC
Жуи 1131 1134 PER
Диндин 1137 1143 PER
Москву 1446 1452 LOC
Пекина 1495 1501 LOC
Фауна Китая 1536 1547 ORG
Московском зоопарке 1551 1570 ORG
России 1596 1602 LOC
Владимир Путин 1603 1617 PER
Китайской Народной Республики 1633 1662 LOC
Си Цзиньпин 1663 1674 PER
